In [1]:
import numpy as np
import cv2
import os
import time
import math
import imutils



In [7]:
def Setup(yolo):
    global net,LABELS,ln,COLORS
    weights = os.path.sep.join([yolo, "yolov3.weights"])
    config = os.path.sep.join([yolo, "yolov3.cfg"])
    labelsPath = os.path.sep.join([yolo, "coco.names"])
    LABELS = open(labelsPath).read().strip().split("\n")
    np.random.seed(42)
    #COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	#dtype="uint8")
    #loading the yolo network
    net = cv2.dnn.readNetFromDarknet(config, weights)
    ln = net.getLayerNames()#it will give the layers name
  #  254 ['conv_0', 'bn_0', 'relu_0', 'conv_1', 'bn_1', 'relu_1', 'conv_2', 'bn_2',
#'relu_2', 'conv_3', 'bn_3', 'relu_3', 'shortcut_4', 'conv_5', 'bn_5', 'relu_5',
#'conv_6', 'bn_6', 'relu_6', 'conv_7', 'bn_7', 'relu_7', 'shortcut_8', 'conv_9',
#'bn_9', 'relu_9', 'conv_10', 'bn_10', 'relu_10', 'shortcut_11', 'conv_12', 'bn_12',
#...
#getUnconnectedOutLayers will give the locations for unconnected layers.
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    

In [35]:
  
    
def Process_Frame(image):
    global processedImg,box_line
    (H, W) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    starttime = time.time()
    layerOutputs = net.forward(ln)
    #print(len(layerOutputs))
    stoptime = time.time()
    print("Video is Getting Processed at {:.4f} seconds per frame".format((stoptime-starttime))) 
    class_lst=[]
    boxes = []
    confidences = []
    
    for output in layerOutputs:
      #print(len(output))
        
   # loop over each of the detections
      for detection in output:
        # extract the class ID and confidence (i.e., probability)
        # of the current object detection
          scores = detection[5:]
          classID = np.argmax(scores)
          confidence = scores[classID]

          # filter detections by (1) ensuring that the object
          # detected was a person and (2) that the minimum
          # confidence is met
          if((classID == LABELS.index("car") or classID==LABELS.index("truck") or classID==LABELS.index("bus") or classID==LABELS.index("motorbike"))):
            if confidence>0.5:
                # scale the bounding box coordinates back relative to
                # the size of the image, keeping in mind that YOLO
                # actually returns the center (x, y)-coordinates of
                # the bounding box followed by the boxes' width and
                # height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates,
                # centroids, and confidences
                boxes.append([x, y, int(width), int(height)])
                #centroids.append((centerX, centerY))
                confidences.append(float(confidence))
                class_lst.append(classID)
                

     # apply non-maxima suppression to suppress weak, overlapping
# bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)
    print(class_lst)

# ensure at least one detection exists
    if len(idxs) > 0:
# loop over the indexes we are keeping

      for i in idxs.flatten():
      # extract the bounding box coordinates
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        color = (255, 0, 0) 
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        if LABELS[class_lst[i]]=="car":
          text = " {}: Toll to pay - 100 ".format(LABELS[class_lst[i]])
          cv2.putText(image, text, (x, y +160 ), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)
        elif LABELS[class_lst[i]]=="truck":
            text = " {}: Toll to pay -350  ".format(LABELS[class_lst[i]])
            cv2.putText(image, text, (x, y+160), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)
        elif LABELS[class_lst[i]]=="bus":
            text = " {}: Toll to pay -350  ".format(LABELS[class_lst[i]])
            cv2.putText(image, text, (x, y+160), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)
        else:
            text = " {}: Toll to pay - 350  ".format(LABELS[class_lst[i]])
            cv2.putText(image, text, (x, y+160), cv2.FONT_HERSHEY_SIMPLEX,0.5, color, 2)
        
# show the output image
    cv2.imshow("Image", image)

      

In [ ]:

yolo = "trivial/"
cap = cv2.VideoCapture(r"H:\stitch.avi")
#fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'XVID') 
out = cv2.VideoWriter('Output.avi', fourcc, 20.0, (640, 480)) 

cnt=0
time1 = time.time()

while True:
  print("hey")
  ret,frame=cap.read()
  print("he")
  print(ret)

  if ret==True:
    frame = imutils.resize(frame, width=700)
    print("key")

    Setup(yolo)
    print("hey")
    Process_Frame(frame)
    
    
   
    cnt += 1
    
    if cv2.waitKey(1) & 0xFF == ord("q"): break
  else:
         break
    
    

    

cap.release()
cv2.destroyAllWindows()



hey
he
True
key
hey
Video is Getting Processed at 2.2461 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.8691 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.7746 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.8279 seconds per frame
[2]
hey
he
True
key
hey
Video is Getting Processed at 1.8978 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.9604 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.8504 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.7233 seconds per frame
[2]
hey
he
True
key
hey
Video is Getting Processed at 1.8215 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.7345 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.7890 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.8575 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.7210 secon

hey
Video is Getting Processed at 1.8338 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Getting Processed at 2.0045 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Getting Processed at 1.7905 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Getting Processed at 1.8354 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Getting Processed at 1.7192 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Getting Processed at 1.8415 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Getting Processed at 1.6965 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Getting Processed at 1.7522 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Getting Processed at 1.7362 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Getting Processed at 1.7848 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Getting Processed at 1.7488 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Getting Processed at 1.7380 seconds per frame
[7, 7]
hey
he
True
key
hey
Video is Get

Video is Getting Processed at 1.7343 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.9622 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 2.0836 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7879 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7424 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7199 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7513 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7250 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7547 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7349 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7355 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7509 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8287 seconds per fra

hey
Video is Getting Processed at 2.3088 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7652 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7731 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7698 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7118 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7426 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8563 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7770 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8075 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7846 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7369 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7836 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7785 seconds per

hey
Video is Getting Processed at 1.7197 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7639 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8053 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7812 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7733 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7196 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7729 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7226 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7691 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7274 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7275 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7809 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7956 seconds per

...

hey
Video is Getting Processed at 1.7626 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7633 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7720 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8471 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7616 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7802 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7855 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7357 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8460 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 2.0078 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8032 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7442 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8423 seconds per

hey
Video is Getting Processed at 1.8025 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7935 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7732 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7601 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7681 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7731 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7367 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7666 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7447 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7643 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7210 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7886 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8877 seconds per

hey
Video is Getting Processed at 1.7518 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7203 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7582 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7698 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8297 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.9239 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8181 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 2.2156 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.8456 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7663 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7414 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7682 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7397 seconds per

hey
Video is Getting Processed at 1.7921 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7581 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7340 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7316 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7568 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7317 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7609 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7637 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7488 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7717 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7636 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7505 seconds per frame
[7]
hey
he
True
key
hey
Video is Getting Processed at 1.7281 seconds per

hey
Video is Getting Processed at 1.9268 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.8065 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.9121 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 2.3745 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 2.3512 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 2.7876 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.7695 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 2.5971 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 2.1933 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.9537 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.8521 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 2.1232 seconds per frame
[]
hey
he
True
key
hey
Video is Getting Processed at 1.9999 seconds per frame
[]
he